# In-Class Assignment — Data Preprocessing & Cleaning (Text)  
**Time:** 20 minutes  |  **Points:** 10  

## Instructions
- This is an individual in-class assignment.  
- Write your code **inside each answer cell**.  
- Print the required outputs.  
- Submit your GitHub/Colab link as instructed by the instructor.


You are given a small dataset of customer support messages as a **TAB-separated text file**:  
- `support_messages.txt`

You will download this file from **Canvas** and upload it to your **Google Colab** notebook.

**How to upload it to your Google Colab notebook?**

1. Download `support_messages.txt` from Canvas.
3. In **the left sidebar**, click the **Files** icon (folder).  
4. Click **Upload** and select `support_messages.txt`.

6. RightAfter uploading, the file will appear in the Colab file list on the left.

6. Right-click the file, copy its path, and paste it into the FILE_PATH variable in Q1.

7. Run Q1 to load the dataset.



> Important: Keep the file name exactly as `support_messages.txt`.


## Questions (Total = 10 points)

### Q1 (1 point) — Load the dataset
Load the TAB-separated file into a pandas DataFrame with columns: `id`, `message`.  
Print: **(a)** `df.shape`, **(b)** `df.head(3)`.

### Q2 (3 points) — Descriptive columns
Add these columns for each message and print the full DataFrame:
- `word_count`: number of words  
- `char_count`: number of characters  
- `num_count`: number of digits (0–9)  
- `upper_word_count`: number of ALL-CAPS words (e.g., `"WHY"`, `"DAMAGED"`)  

### Q3 (3 points) — Clean text
Build a `clean_text(text)` function and create a new column `clean` with these steps **in order**:
1) lowercase  
2) remove punctuation/symbols (keep letters/numbers/spaces)  
3) remove English stopwords (use **nltk** or **sklearn** list)  
4) remove extra spaces  

Print the **original** message and **clean** version for rows `id=1` and `id=4`.

### Q4 (2 points) — Regex extraction
Using RegEx, extract and create two new columns:
- `order_id`: first occurrence of pattern `ORD-####` (case-insensitive; `ord-1060` is valid)  
- `email`: first email address if present (otherwise `None`/`NaN`)  

Print: `id`, `order_id`, `email` for all rows.

### Q5 (1 point) — TF-IDF keywords
Using the `clean` column, compute **TF-IDF** for the messages and print the **top 5 keywords** with the highest **average TF-IDF** across documents.


In [8]:
# Setup (run this cell first)
import re
import pandas as pd


## Q1 (1 point) — Answer below

In [9]:
# Q1 — ANSWER CELL
FILE_PATH = "support_messages.txt"  # Update path if needed

# Load the TAB-separated file into df
df = pd.read_csv('/Users/jeevanchandaka/Downloads/support_messages.txt', sep="\t")

# (a) print shape
print("df.shape:", df.shape)

# (b) print first 3 rows
print(df.head(3))


df.shape: (8, 2)
   id  \
0   1   
1   2   
2   3   

                                                                     message  
0  Hi!! My ORDER is late :(  Order# ORD-1042. Email me at sara.Ali@gmail.com  
1    Refund please!!! I was charged 2 times... invoice ORD-1042 and ORD-1043  
2                                Great service, thanks! arrived in 2 days :)  


## Q2 (3 points) — Answer below

In [10]:
## Q2 — ANSWER CELL

# word_count: number of words
df["word_count"] = df["message"].str.split().apply(len)

# char_count: number of characters
df["char_count"] = df["message"].str.len()

# num_count: number of digits (0-9)
df["num_count"] = df["message"].str.count(r"\d")

# upper_word_count: number of ALL-CAPS words
df["upper_word_count"] = df["message"].apply(
    lambda text: sum(1 for word in text.split() if word.isupper() and word.isalpha())
)

# Display full DataFrame
pd.set_option('display.max_colwidth', 80)
print(df.to_string(index=False))


 id                                                                      message  word_count  char_count  num_count  upper_word_count
  1    Hi!! My ORDER is late :(  Order# ORD-1042. Email me at sara.Ali@gmail.com          12          73          4                 1
  2      Refund please!!! I was charged 2 times... invoice ORD-1042 and ORD-1043          11          71          9                 1
  3                                  Great service, thanks! arrived in 2 days :)           8          43          1                 0
  4                      WHY is my package DAMAGED??? tracking says delivered...           8          55          0                 1
  5          Need to change address: 7421 Frankford Rd Apt 1232, Dallas TX 75252          12          67         13                 1
  6                   Support ticket: ORD-1050. Call me at (469) 555-0182 ASAP!!           9          58         14                 0
  7 I can’t login— password reset link not working. email: meh

## Q3 (3 points) — Answer below

In [11]:
# Q3 — ANSWER CELL
import nltk
nltk.download("stopwords", quiet=True)
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words("english"))

def clean_text(text: str) -> str:
    # Step 1: Lowercase
    text = text.lower()
    # Step 2: Remove punctuation/symbols (keep letters, numbers, spaces)
    text = re.sub(r"[^a-z0-9 ]", " ", text)
    # Step 3: Remove English stopwords
    tokens = text.split()
    tokens = [t for t in tokens if t not in STOPWORDS]
    text = " ".join(tokens)
    # Step 4: Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Create clean column
df["clean"] = df["message"].apply(clean_text)

# Print original and clean for id=1 and id=4
for row_id in [1, 4]:
    row = df[df["id"] == row_id].iloc[0]
    print(f"--- id={row_id} ---")
    print(f"Original : {row['message']}")
    print(f"Clean    : {row['clean']}")
    print()

--- id=1 ---
Original : Hi!! My ORDER is late :(  Order# ORD-1042. Email me at sara.Ali@gmail.com
Clean    : hi order late order ord 1042 email sara ali gmail com

--- id=4 ---
Original : WHY is my package DAMAGED??? tracking says delivered...
Clean    : package damaged tracking says delivered



## Q4 (2 points) — Answer below

In [12]:
# Q4 — ANSWER CELL

# order_id: first occurrence of ORD-#### (case-insensitive)
df["order_id"] = df["message"].apply(
    lambda text: (re.search(r"ORD-\d{4}", text, re.IGNORECASE) or type('', (), {'group': lambda self: None})()).group(0)
    if re.search(r"ORD-\d{4}", text, re.IGNORECASE) else None
)

# email: first email address if present
EMAIL_PATTERN = r"[A-Za-z0-9._%+\-]+@[A-Za-z0-9.\-]+\.[A-Za-z]{2,}"
df["email"] = df["message"].apply(
    lambda text: re.search(EMAIL_PATTERN, text).group(0)
    if re.search(EMAIL_PATTERN, text) else None
)

# Print id, order_id, email for all rows
print(df[["id", "order_id", "email"]].to_string(index=False))

 id order_id                 email
  1 ORD-1042    sara.Ali@gmail.com
  2 ORD-1042                  None
  3     None                  None
  4     None                  None
  5     None                  None
  6 ORD-1050                  None
  7     None mehri.sattari@unt.edu
  8 ord-1060                  None


## Q5 (1 point) — Answer below

In [13]:
# Q5 — ANSWER CELL
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Fit TF-IDF on clean column
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["clean"])

# Compute average TF-IDF per term across documents
avg_tfidf = np.asarray(tfidf_matrix.mean(axis=0)).flatten()
terms = vectorizer.get_feature_names_out()

# Get top 5 keywords
top5_idx = avg_tfidf.argsort()[::-1][:5]
print("Top 5 TF-IDF Keywords:")
for idx in top5_idx:
    print(f"  {terms[idx]:<20} {avg_tfidf[idx]:.4f}")

Top 5 TF-IDF Keywords:
  ord                  0.1329
  order                0.0990
  1042                 0.0685
  email                0.0658
  says                 0.0559


## Grading Checklist
- Q1: correct load + prints  
- Q2: correct counts  
- Q3: cleaning follows the required order + prints for id=1 and id=4  
- Q4: regex extraction works (case-insensitive `ORD-####` and emails)  
- Q5: prints 5 keywords + their scores (rounding is fine)
